In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import agent as ag
import df_helpers as dfh
import model as ml

from matplotlib import style

from google_window import WindowGenerator
from model import get_lstm_model
# style.use('dark_background')

import importlib
importlib.reload(ag)
importlib.reload(ml)
importlib.reload(dfh)

In [ ]:
# Open no1_train.csv and no1_validation.csv
original_df_train = pd.read_csv('no1_train.csv')
original_df_test = pd.read_csv('no1_validation.csv')

# Make copies
df_train = original_df_train.copy()
df_test = original_df_test.copy()

In [ ]:
df_train

In [ ]:
df_test.head()

In [ ]:
# Convert timestamps in "start_time" to seconds
df_train['start_time_seconds'] = pd.to_datetime(df_train['start_time'])
df_train['start_time_seconds'] = df_train['start_time_seconds'].apply(lambda x: x.timestamp())

# Do the same to df_test
df_test['start_time_seconds'] = pd.to_datetime(df_test['start_time'])
df_test['start_time_seconds'] = df_test['start_time_seconds'].apply(lambda x: x.timestamp())

## Exploratory Data Analysis

First let's look at the data and see if we notice any outliers that might not correlate well with the overall trend of the data

In [ ]:
df_train.describe().transpose()

Seeing as river has absolutely all values equal to 0, this is not a good feature to use. We therefore drop it because
it gives no additional information. We will also drop sysreg since it seems this feature too is very monotonous, and does not yield much information.

In [ ]:
df_train = df_train.drop(['river'], axis=1)
df_test = df_test.drop(['river'], axis=1)

df_train = df_train.drop(['sys_reg'], axis=1)
df_test = df_test.drop(['sys_reg'], axis=1)

Let's visualize the data in several plots to see if we instinctively can see anything that doesn't add up

In [ ]:
# Get column names from df_train except start_time
cols = df_train.columns.drop('start_time')
plot_features = df_train[cols]
plot_features.index = df_train['start_time']
_ = plot_features.plot(subplots=True, figsize=(20, 12))

# Do the same for df_test
plot_features = df_test.drop("start_time", axis=1)[cols]
plot_features.index = df_test['start_time']
_ = plot_features.plot(subplots=True, figsize=(20, 12))

In [ ]:
# Plot two different plots; one for df_train, one for df_test with feature "y"
plot_features = df_train[['y']]
plot_features.index = df_train['start_time']
_ = plot_features.plot(subplots=True, figsize=(20, 12))

# Do the same for df_test, but in a new plot 
plot_features = df_test[['y']]
plot_features.index = df_test['start_time']
_ = plot_features.plot(subplots=True, figsize=(20, 12))


There are some spikes in both datasets, especially the validation data, that seem rather inconsistent. Let's replace them with values based on the mean of the "y" value for other datapoints that have similar "total" feature.

In [ ]:
df1 = dfh.get_rows_between(df_train, 'total', 1500, 1550)
df2 = dfh.get_rows_between(df1, 'y', -2900, 2900)
mean_val = df2["y"].mean()
q = (df_train["y"] > 1500) | (df_train["y"] < -1500)
n_clamps = df_train.loc[q].shape[0]
print(f"Percentage of rows clamped in training: {round(n_clamps / df_train.shape[0], 2) * 100}%")
df_train.loc[q, "y"] = mean_val

# Do the same for df_test
df1 = dfh.get_rows_between(df_test, 'total', 1500, 1550)
df2 = dfh.get_rows_between(df1, 'y', -2900, 2900)
mean_val = df2["y"].mean()
q = (df_test["y"] > 1500) | (df_test["y"] < -1500)
n_clamps = df_test.loc[q].shape[0]
print(f"Percentage of rows clamped in testing: {round(n_clamps / df_test.shape[0], 2) * 100}%")
df_test.loc[q, "y"] = mean_val

Let's see if there are any NaN-values present

In [ ]:
# Check for NaN values
df_train.isnull().sum()

## Feature engineering
In this section, we will look at how we can manipulate the data in the dataset in order to better suit it for model prediction. This will include modifying existing features, and introducing new ones.

First we'll implement the required feature: previous_y

Second we'll look at frequencies.
The nature of demand on the power grid is highly dependent on two key factors:
* The time of day
* The time of year

Seeing as the price of electricity has been a heated debate for the past half-year due to environmental and seasonal changes, this might be a good place to start.

In [ ]:
day = 24*60*60
year = (365.2425)*day

df_train['Day sin'] = np.sin(df_train['start_time_seconds'] * (2 * np.pi / day))
df_train['Day cos'] = np.cos(df_train['start_time_seconds'] * (2 * np.pi / day))
df_train['Year sin'] = np.sin(df_train['start_time_seconds'] * (2 * np.pi / year))
df_train['Year cos'] = np.cos(df_train['start_time_seconds'] * (2 * np.pi / year))

# Do the same, but for df_test
df_test['Day sin'] = np.sin(df_test['start_time_seconds'] * (2 * np.pi / day))
df_test['Day cos'] = np.cos(df_test['start_time_seconds'] * (2 * np.pi / day))
df_test['Year sin'] = np.sin(df_test['start_time_seconds'] * (2 * np.pi / year))
df_test['Year cos'] = np.cos(df_test['start_time_seconds'] * (2 * np.pi / year))


Let's also add simple bucket-indicators for the time of day since this massively impacts the use of the power grid.
We will use the following buckets for the feature 'time_of_day':

* 0 = early morning. Times between 06:00 and 09:00
* 1 = late morning. Times between 09:00 and 12:00
* 2 = early day. Times between 12:00 and 15:00
* 3 = evening. Times between 15:00 and 18:00
* 4 = late evening. Times between 18:00 and 21:00
* 5 = early night. Times between 21:00 and 00:00
* 6 = night time. Times between 00:00 and 06:00

In [ ]:
df_train['hours'] = pd.to_datetime(df_train['start_time']).apply(lambda x: x.hour)
df_train['time_of_day'] = 0
# If time_of_day is between 6 and 8, set it to 0
df_train.loc[(df_train['hours'] >= 6) & (df_train['hours'] <= 8), 'time_of_day'] = 0
# If between 9 and 11, set it to 1
df_train.loc[(df_train['hours'] >= 9) & (df_train['hours'] <= 11), 'time_of_day'] = 1
# If between 12 and 14, set it to 2
df_train.loc[(df_train['hours'] >= 12) & (df_train['hours'] <= 14), 'time_of_day'] = 2
# If between 15 and 17, set it to 3
df_train.loc[(df_train['hours'] >= 15) & (df_train['hours'] <= 17), 'time_of_day'] = 3
# If between 18 and 20, set it to 4
df_train.loc[(df_train['hours'] >= 18) & (df_train['hours'] <= 20), 'time_of_day'] = 4
# If between 21 and 23, set it to 5
df_train.loc[(df_train['hours'] >= 21) & (df_train['hours'] <= 23), 'time_of_day'] = 5
# If between 0 and 5, set it to 6
df_train.loc[(df_train['hours'] >= 0) & (df_train['hours'] <= 5), 'time_of_day'] = 6
df_train.drop(columns='hours', inplace=True)

# Do the same for df_test
df_test['hours'] = pd.to_datetime(df_test['start_time']).apply(lambda x: x.hour)
df_test['time_of_day'] = 0
df_test.loc[(df_test['hours'] >= 6) & (df_test['hours'] <= 8), 'time_of_day'] = 0
df_test.loc[(df_test['hours'] >= 9) & (df_test['hours'] <= 11), 'time_of_day'] = 1
df_test.loc[(df_test['hours'] >= 12) & (df_test['hours'] <= 14), 'time_of_day'] = 2
df_test.loc[(df_test['hours'] >= 15) & (df_test['hours'] <= 17), 'time_of_day'] = 3
df_test.loc[(df_test['hours'] >= 18) & (df_test['hours'] <= 20), 'time_of_day'] = 4
df_test.loc[(df_test['hours'] >= 21) & (df_test['hours'] <= 23), 'time_of_day'] = 5
df_test.loc[(df_test['hours'] >= 0) & (df_test['hours'] <= 5), 'time_of_day'] = 6
df_test.drop(columns='hours', inplace=True)

Now we will apply the same logic for time_of_week, however we will have to use different buckets. We'll simply add one bucket for each day of the week, i.e. 0 for monday, 1 for tuesday, etc..
We will also add an arbitrary feature called 'weekend', which will apply to saturday and sunday (1 for 'weekend' = True, else 0)

In [ ]:
df_train['time_of_week'] = pd.to_datetime(df_train['start_time']).apply(lambda x: x.weekday())
# If time_of_week = 5 or 6, set feature 'weekend' to 1. Else set to 0
df_train['weekend'] = 0
df_train.loc[(df_train['time_of_week'] == 5) | (df_train['time_of_week'] == 6), 'weekend'] = 1

# Do the same for df_test
df_test['time_of_week'] = pd.to_datetime(df_test['start_time']).apply(lambda x: x.weekday())
df_test['weekend'] = 0
df_test.loc[(df_test['time_of_week'] == 5) | (df_test['time_of_week'] == 6), 'weekend'] = 1

Finally, we get to time_of_year. Here we will simply use the different months that the dates correspond to. Using this, we will also add a feature called 'season' to explicitly state whether we are in the summer, winter, autumn, or spring.
We will bucket the season as follows using [this](https://snl.no/%C3%A5rstider) definition:

* Spring will go from March through May
* Summer will start in June, and end with (including) August
* Autumn then starts from September, and runs until November
* Finally, winter runs from December through February

In [ ]:
df_train['time_of_year'] = pd.to_datetime(df_train['start_time']).apply(lambda x: x.month)
df_train['season'] = 0
# If time_of_year is between 3 and 5, set it to 0
df_train.loc[(df_train['time_of_year'] >= 3) & (df_train['time_of_year'] <= 5), 'season'] = 0
# If between 6 and 8, set it to 1
df_train.loc[(df_train['time_of_year'] >= 6) & (df_train['time_of_year'] <= 8), 'season'] = 1
# If between 9 and 11, set it to 2
df_train.loc[(df_train['time_of_year'] >= 9) & (df_train['time_of_year'] <= 11), 'season'] = 2
# If between 12 and 2, set it to 3
df_train.loc[df_train['time_of_year'] == 12, 'season'] = 3
df_train.loc[df_train['time_of_year'] <= 2, 'season'] = 3

# Do the same for df_test
df_test['time_of_year'] = pd.to_datetime(df_test['start_time']).apply(lambda x: x.month)
df_test['season'] = 0
df_test.loc[(df_test['time_of_year'] >= 3) & (df_test['time_of_year'] <= 5), 'season'] = 0
df_test.loc[(df_test['time_of_year'] >= 6) & (df_test['time_of_year'] <= 8), 'season'] = 1
df_test.loc[(df_test['time_of_year'] >= 9) & (df_test['time_of_year'] <= 11), 'season'] = 2
df_test.loc[df_test['time_of_year'] == 12, 'season'] = 3
df_test.loc[df_test['time_of_year'] <= 2, 'season'] = 3

We don't need the features that give us the time in raw values anymore, therefore we remove them.

In [ ]:
df_train.drop(columns=['start_time', 'start_time_seconds'], inplace=True)
df_test.drop(columns=['start_time', 'start_time_seconds'], inplace=True)

In [ ]:
df_train.head()

Finally, we add one-hot encoding of these variables to make the dataframe more sparse

In [ ]:
# One-hot encode the columns with time_of_day, time_of_week, time_of_year, and season
one_hot_feats = ['time_of_day', 'time_of_week', 'time_of_year', 'season']
df_train = pd.get_dummies(df_train, columns=one_hot_feats)
df_test = pd.get_dummies(df_test, columns=one_hot_feats)

In [ ]:
# The following features did not get added to df_test since they are not observed, therefore they must be added manually
diff_feats = [a for a in df_train.columns if a not in df_test.columns]
df_test[diff_feats] = 0

Let's now just add a bunch of lagged variables. This is mainly because the model seems to heavily weight the data from the very near past, so let's just try to reinforce this

In [ ]:
# Add a column which shifts 'total' by 24 hours
df_train = dfh.add_shift(df_train, 'total', 24)
df_train = dfh.add_shift(df_train, 'total', 12)
df_train = dfh.add_shift(df_train, 'total', 6)
# Do the same for feature 'flow'
df_train = dfh.add_shift(df_train, 'flow', 24)
df_train = dfh.add_shift(df_train, 'flow', 12)
df_train = dfh.add_shift(df_train, 'flow', 6)

# Do the same for df_test
df_test = dfh.add_shift(df_test, 'total', 24)
df_test = dfh.add_shift(df_test, 'total', 12)
df_test = dfh.add_shift(df_test, 'total', 6)
# Do the same for feature 'flow'
df_test = dfh.add_shift(df_test, 'flow', 24)
df_test = dfh.add_shift(df_test, 'flow', 12)
df_test = dfh.add_shift(df_test, 'flow', 6)

We will now add the feature 'previous_y', which is the imbalance from the previous timestep. We will then use this value to add a rolling average for the last 24, 12, 6, and 3 hours, meaning we get 4 additional features that measure the average 'previous_y' value based on those previous hour segments.

Note that unlike ```add_shift```, ```add_shifted_target``` shifts by 1 hard-coded index, rather than a variable number of hours (this is why they are 2 different functions).

In [ ]:
df_train = dfh.add_shifted_target(df_train, 'y')
df_test = dfh.add_shifted_target(df_test, 'y')

df_train = dfh.add_rolling_avg(df_train, 'previous_y', hours=24)
df_train = dfh.add_rolling_avg(df_train, 'previous_y', hours=12)
df_train = dfh.add_rolling_avg(df_train, 'previous_y', hours=6)
df_train = dfh.add_rolling_avg(df_train, 'previous_y', hours=3)

#dftest
df_test = dfh.add_rolling_avg(df_test, 'previous_y', hours=24)
df_test = dfh.add_rolling_avg(df_test, 'previous_y', hours=12)
df_test = dfh.add_rolling_avg(df_test, 'previous_y', hours=6)
df_test = dfh.add_rolling_avg(df_test, 'previous_y', hours=3)

In [ ]:
RESOLUTION = 5
N_PREV = 24
START_INDEX = 300
BATCH_SIZE = 64
TARGET = 'y'
EPOCHS = 20
agent = ag.Agent(
    min_scale=-1,
    max_scale=1,
    resolution=RESOLUTION,
    n_prev=N_PREV,
    start_index=START_INDEX,
    batch_size=BATCH_SIZE,
    target=TARGET,
    verbose=True,
    model=get_lstm_model(),
    filepath='./models/LSTM_model_1',
)

In [ ]:
agent.fit_scalers(df_train)
df_train = agent.transform(df_train)
df_test = agent.transform(df_test)

In [ ]:
df_test.head()

In [ ]:
df_train.head()

In [ ]:
assert(all([a in df_train.columns for a in df_test.columns]))

In [ ]:
# Here one can drop features to see if the model improves with less features
# df_train.drop(columns=['Day sin', 'Day cos', 'Year sin', 'Year cos'], inplace=True)

In [ ]:
history = agent.train(df_train, epochs=EPOCHS)

In [ ]:
loss_per_epoch = history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
x_valid = df_test.drop("y", axis=1)
x_valid.head()

In [ ]:
y_pred = agent.predict_n_timesteps(df=x_valid, n_timesteps=1000, replace=True)
y_true = agent.scalers['y'].inverse_transform(df_test['y'].to_numpy().reshape(-1,1))
agent.visualize_results(y_true, y_pred, n_timesteps=1000)